In [6]:
from transformers import AutoTokenizer, MixtralModel, MixtralConfig
config = MixtralConfig.from_pretrained("hf-internal-testing/Mixtral-tiny")
config.output_router_logits = True
config.output_attentions = True
model = MixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny", config=config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)

MixtralModel is using MixtralSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [12]:
for i in x:
    print(i)

last_hidden_state
past_key_values
attentions
router_logits


In [8]:
x.attentions

(tensor([[[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.4987, 0.5013, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.3037, 0.1833, 0.5130,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0617, 0.0380, 0.0840,  ..., 0.0823, 0.0000, 0.0000],
           [0.0878, 0.1733, 0.0588,  ..., 0.0614, 0.0586, 0.0000],
           [0.0648, 0.1383, 0.1151,  ..., 0.0713, 0.0706, 0.0813]],
 
          [[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.5045, 0.4955, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.3550, 0.2264, 0.4186,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0694, 0.0734, 0.1772,  ..., 0.0526, 0.0000, 0.0000],
           [0.0404, 0.0423, 0.1207,  ..., 0.1065, 0.0829, 0.0000],
           [0.0542, 0.0540, 0.0561,  ..., 0.0910, 0.0697, 0.0517]],
 
          [[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.4108, 0.5892, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.3897, 0.36

In [1]:
from transformers import AutoTokenizer, FlaxMixtralModel, MixtralConfig

config = MixtralConfig.from_pretrained("hf-internal-testing/Mixtral-tiny")
config.output_router_logits = True
config.output_attentions = True
# config.past_key_value = True
model = FlaxMixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny", config=config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

y = model(**inputs)

/home/adrien/Documents/flax/FlaxMixtral/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Some weights of the model checkpoint at hf-internal-testing/Mixtral-tiny were not used when initializing FlaxMixtralModel: {('lm_head', 'kernel')}
- This IS expected if you are initializing FlaxMixtralModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxMixtralModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some of the wei

In [2]:
y.router_logits

In [ ]:
y.last_hidden_state

In [ ]:
for i in y:
    print(i)

In [ ]:
from transformers import AutoTokenizer, FlaxMixtralForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")
model = FlaxMixtralForCausalLM.from_pretrained("hf-internal-testing/Mixtral-tiny")

inputs = tokenizer("Hello, my dog is cute", return_tensors="jax")
outputs = model(**inputs)

# retrieve logts for next token
# next_token_logits = outputs.logits[:, -1]
# next_token_logits

In [ ]:
for i in outputs:
    print(i)